# Vorhersage von Kauf-, Halte- und Verkaufssingale für Aktien

Im Finanzhandel ist es entscheidend, fundierte Entscheidungen auf Basis von Daten zu treffen. Mit der Analyse von Aktienkursen können Handelsstrategien entwickelt werden, die dabei helfen, zu entscheiden, wann man eine Aktie kaufen, verkaufen oder halten sollte. In diesem Projekt werden zwei verschiedene Ansätze verwendet, um solche Handelsentscheidungen zu treffen.
### Ziel
Das Ziel dieses Projekts ist es, zwei Modelle zu entwickeln, die Handelsentscheidungen auf Basis von historischen Aktienkursdaten vorhersagen können.

1.   Technische Indikatoren-Modell: Dieses Modell nutzt spezielle Kennzahlen, die aus den historischen Kursdaten berechnet werden, um Vorhersagen zu treffen.
2.   Rohdaten-Modell: Dieses Modell verwendet die grundlegenden Kursdaten wie Öffnungskurs, Höchstkurs, Tiefstkurs, Schlusskurs und Handelsvolumen, um Entscheidungen zu treffen.

### Vorgehen

Zuerst werden die Daten aufbereitet und in Trainings- und Testsets unterteilt. Dann werden die Modelle trainiert und auf ihre Genauigkeit getestet. Schließlich werden Vorhersagen für das Jahr 2022 erstellt und simuliert, wie sich diese Handelsstrategien in der Praxis bewähren würden.

Das Projekt zeigt, wie unterschiedliche Ansätze zur Vorhersage von Handelssignalen eingesetzt werden können und bietet Einblicke in die mögliche Leistung dieser Modelle.

## 1. Notwendige Vorbereitung!

### 1.1 Importieren der benötigten Bibliotheken

In [ ]:
import random
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta

Durch das importieren von **yfinance** die den alias namen **yf** können die Daten für unterschiedliche Aktien auf der Seite **Yahoo Finance** eingelesen werden.

Die importierung von **pandas** (alias namen **pd**) und **numpy** (alias namen **np**) können die eingelesenen Daten Manipuliert werden beziehungsweise auch Datan Analysiert werden.

Durch **sklearn.ensemble (RandomForestClassifier)** wird der Random Forest Algorithmus importiert der mehrere Entscheidungsbäume kombinieren kann, um eine Klassifizierte Vohersage zu treffen. In diesem Fahll Kauf-, Verkaufs- und Haltessignale für Aktien die für den Ansatz mit den Technischen Indikatoren verwendet wird.

Durch **sklearn.linear_model (LogisticRegression)** wird das Algorithmus logistische Regression hinzugefügt die dabei hilft auf Basis der Rohdaten von der jeweiligen Aktie einen Kauf-, Verkaufs- und Haltesignale zu erstellen.

Durch **sklearn.metric** wird  **classification_report** und **accuracy_score** importiert:

Mit dem **classfication_report** kann das Klassfikationmodell bewertet werden über die erbrachtete leistung.

durch die **accuracy_score** wird die genauigket des Klassfikationmodell errechnet.

Durch die **sklearn.preprocessing** wird der **StandardScaler** importiert umnumerische Features zu skalieren das heißt der Mittelwert wird als 0 und die Standardabweichung als 1 definiert.

Mit dem **Datetime**, werden Tage Stunden Minuten bzw. auch Datums dargestellt.

### 1.2 definierung der mehrfach verwendete Variabeln

In [ ]:
def initialize_globals():
    """
    Initialisiert und gibt die globalen Parameter für das Training und Testen zurück.
    """
    random_seed = 42
    random.seed(random_seed)
    np.random.seed(random_seed)

    train_start = '2010-01-01'
    train_end = '2020-12-31'
    test_start = '2021-01-01'

    threshold = 2

    features = ['MA5', 'MA10', 'RSI', 'UpperBand', 'LowerBand', 'Momentum', 'ROC', 'Volatility']
    raw_features = ['Open', 'High', 'Low', 'Close', 'Volume']
    return train_start, train_end, test_start, threshold, features, raw_features, random_seed

Die Methode **initialize_globals()** wird verwendet, um Variablen zu definieren die häufiger verwendet werdem .

```
    train_start = '2010-01-01'
    train_end = '2020-12-31'
    test_start = '2021-01-01'
```
Diese drei Variabeln teilen die Daten auf. Einmal für Trainingszeitraum und einmal für den Testzeitraum. Die Vorteile hierbei ist das das Modell realistischer agieren kann. Desweitern kann Zeitabhängige Muster entwickelt werden, das Muster erlernen.


```
threshold = 2
```
Mit dem Threshold-Wert auch Schwellenwert genannt können wir festlegen, wie stark der Preis eines Akties verändern muss, um diesen als einen Handelssignal generieren zulassen. Konkret gesagt, ist wenn die Prozentualle Preisänderung über diesen threshold Variable liegt wird ein Kaufssignal generiert und wenn die Preisänderung unter threshold liegt wird ein Verkaufssignal generiert



```
features = ['MA5', 'MA10', 'RSI', 'UpperBand', 'LowerBand', 'Momentum', 'ROC', 'Volatility']
```
Mit den features werden die Technischen Indikatoren festgehalten die in unterschiedlichen Methodiken verwendet wird. Dadurch das diese Variable an unterschiedlichen Methoden verwendet wird, ist es für die wartbarkeit einfach diese auch in der initialize_globals() zu definieren.



### 1.3 Vorbereitung der Eingabe vom Nutzer für die Aktie

In [ ]:
# Funktion zur Eingabe des Tickersymbols
def get_ticker():
    while True:
        ticker = input("Bitte geben Sie das Tickersymbol der Aktie ein (Standard ist AAPL): ").strip().upper()

        if not ticker:
          ticker = "AAPL"

        try:
            # Versuche, Informationen über das Tickersymbol abzurufen
            ticker_object = yf.Ticker(ticker).info

            return ticker, ticker_object
        except Exception:
            print("Ungültiges Tickersymbol. Bitte versuchen Sie es erneut.")

In der Methode **get_ticker()** Kann der Nutzer falls erwünscht ist eine X Beliebgie Aktie auswählen (die über yfinance erreichbar ist ) eintragen.


```
        ticker = input("Bitte geben Sie das Tickersymbol der Aktie ein (Standard ist AAPL): ").strip().upper()
        if not ticker:
            return 'AAPL'
```


Falls der Nutzer nichts einträgt wird Standardmäßig die APPLE Aktie verwendet, die den Kürzel AAPL hat.

Falls der Nutzer eine Aktie angibt, wird im folgenden Code versuch  an die Aktien information zu kommen.


```
       try:
           # Versuche, Informationen über das Tickersymbol abzurufen
           yf.Ticker(ticker).info
           return ticker  
       except Exception:
           print("Ungültiges Tickersymbol. Bitte versuchen Sie es erneut.")
```
Im Falle eines Fehlschlags wird der Nutzer aufgefordert bitte erneut eine Aktie anzugeben oder auch nichts anzugeben für die definierte Standard Aktie (Apple). Falls die infomationen abgerufen werden können wird die angegebene Aktie zurückgegeben.



## 2. Datenbeschaffung und - aufbereitung

### 2.1 Datenbeschaffung und Datenanzeige

In [ ]:
def get_stock_data():
    while True:
        ticker, ticker_object = get_ticker()
        start_date = '2010-01-01'
        try:
            data = yf.download(ticker,start = start_date)
            if data.empty:
                raise ValueError("Keine Daten für dieses Tickersymbol gefunden.")
            print("\n")
            print("Aktie: " + ticker_object.get('longName') + " wurde erfolgreich Heruntergeladen")
            print("Anzahl der Daten: " + str(len(data)))
            print("\n")
            return data
        except Exception:
            print("Fehler beim Herunterladen der Daten. Bitte versuchen Sie es erneut.")

In der Methode **get_stock_data()** wird die Methode **get_ticker** aufgerufen, um die gewünschte Aktie zu erhalten.
```
    ticker, ticker_object = get_ticker()
```

Nachdem die gewünschte Aktie eingeholt wurde, werden die benötigten Rohdaten heruntergeladen und in die Variable "data" gespeichert.
```
    try:
        data = yf.download(ticker,start = start_date)
```
Nachfolgend wird geprüft, ob die heruntergeladenen Daten leer sind.

```
if data.empty:
            raise ValueError("Keine Daten für dieses Tickersymbol gefunden.")
```
Falls die Daten Leer sind wird man aufgefordert wieder eine Aktie anzugeben.
Falls die Daten nicht leer sind wird dann der Anfang und das Ende der Datensatzes angezeigt


```
            print("\n")
            print("Aktie: " + ticker_object.get('longName') + " wurde erfolgreich Heruntergeladen")
            print(data.head())
            print(data.tail())
            print("\n")
            return data
```









### 2.2 Berechnungen der Technischen Indikatoren

#### 2.2.1 Berechnung vom Moving Avarage

In [ ]:
def calculate_moving_averages(data):
    data['MA5'] = data['Adj Close'].rolling(window=5).mean()
    data['MA10'] = data['Adj Close'].rolling(window=10).mean()
    print("\n")
    print("Berechnung der Moving Averages MA5 und MA10")
    print("\n")
    return data

In der Methode **calculate_moving_averages(data)** wird der gleitenden Durchschnitt der letzten 5 Tage und der letzten 10 Tage für den bereinigten Schlusskurs (Adj Close) ermittelt.
```
    data['MA5'] = data['Adj Close'].rolling(window=5).mean()
```
Der gleitende Durchschnitt für die letzten 5 Tage hilft für die identifizierung von kurzfristige Kauf- und Verkaufssignale
```
    data['MA10'] = data['Adj Close'].rolling(window=10).mean()
```
Der gleitende Durchschnitt für die letzten 10 Tage gibt an in welche richtung die Aktie sich entwickelt.


Diese zwei gleitende Durchschnitte werden in das DataFrame(data) mit aufgnommen unter der Spalten Namen "MA5" und "MA10" und wird als ganzes DataFrame zurückgegeben ``` return data ```

#### 2.2.2 Berechnung vom Relative Strength Index (RSI)

In [ ]:
def calculate_rsi(data):
    delta = data['Adj Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))
    print("Berechnung des Relative Strength Index(RSI)")
    print("\n")
    return data

Die Methode **calculate_rsi(data)** brechnet den Relative Strength Index die in der technischen Analyse hilft, um den Dynamik und das Momentum einer Aktie zu beurteilen.

Die Formel für **RSI** lautet: ``` RSI = 100 - (100 / (1 + RS)) ```

Die Formel für **RS** lautet: ```RS = Durschnittlicher Gewinn / Durchschnittlicher Verlust```

```
    delta = data['Adj Close'].diff(1)
```
In der Variable delta werden tägliche Preisänderungen gespeichert

```
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
```
In ```gain```
werden die tägliche positive Preisänderungen aus ```delta```
gefiltert, während in ```loss``` die tägliche negative Preisänderung gefiltert wird.
```
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
```
Infolge dessen werden die durchschnittliche Gewinn in ```avg_gain```
 / Verlust in ```avg_loss``` der letzten 14 Tage berechnet.

 Im Anschluss werden diese zwei Variable devidiert (Siehe oben RS Formel) und in der Variable ```rs``` abgelagert
```
    data['RSI'] = 100 - (100 / (1 + rs))
```
Als letztes wird der RSI berechnet in dem die Formel eingesetzt wird.

Der RSI wird im ```DataFrame(data)```
mit aufgnommen unter der Spalten Name "RSI" und wird als ganzes DataFrame zurückgegeben ``` return data ```









#### 2.2.3 Berechnung vom Bollinger Bänds

In [ ]:
def calculate_bollinger_bands(data):
    data['MiddleBand'] = data['Adj Close'].rolling(window=20).mean()
    data['UpperBand'] = data['MiddleBand'] + 2 * data['Adj Close'].rolling(window=20).std()
    data['LowerBand'] = data['MiddleBand'] - 2 * data['Adj Close'].rolling(window=20).std()
    print("Berechnung des Bollinger Bänder")
    print("\n")
    return data



In der Methode ```calculate_bollinger_bands(data)``` werden die Bollinger Bänder für die gegebenen Aktiendaten berechnet. Bollinger Bänder sind ein technischer Indikator, der die Volatilität und potenzielle Trendwenden einer Aktie visualisiert.
```
data['MiddleBand'] = data['Adj Close'].rolling(window=20).mean()
```

Mit dem mittlere Band wird der gleitende Durchschnitt des bereinigten Schlusskurses (Adj Close) über die letzten 20 Tage (window=20) berechnet und in einer neuen Spalte namens MiddleBand im DataFrame gespeichert.
```
data['UpperBand'] = data['MiddleBand'] + 2 * data['Adj Close'].rolling(window=20).std()
```
Hier wird das obere Band berechnet. Es wird zum mittleren Band das Doppelte (2 *) der Standardabweichung des bereinigten Schlusskurses über die letzten 20 Tage addiert. Das Ergebnis wird in der Spalte UpperBand gespeichert.
```
data['LowerBand'] = data['MiddleBand'] - 2 * data['Adj Close'].rolling(window=20).std()
```
Hier wird das untere Band berechnet. Es wird vom mittleren Band das Doppelte der Standardabweichung des bereinigten Schlusskurses über die letzten 20 Tage subtrahiert. Das Ergebnis wird in der Spalte LowerBand gespeichert.

```return data```
 Die Funktion gibt den aktualisierten DataFrame zurück, der nun die drei neuen Spalten für die Bollinger Bänder enthält.


#### 2.2.4 Berechnung von weiteren Indkatoren

In [ ]:
def calculate_additional_indicators(data):
    data['Momentum'] = data['Adj Close'] - data['Adj Close'].shift(4)
    data['ROC'] = data['Adj Close'].pct_change(periods=10) * 100
    data['PriceChange'] = data['Adj Close'].pct_change() * 100
    data['Volatility'] = data['Adj Close'].rolling(window=10).std()
    print("Berechnung weiterer Indikatoren")
    print("\n")
    return data

In der Methode ```calculate_additional_indicators(data)``` werden weitere technische Indikatoren berechnet, die für die Analyse und Vorhersage von Aktienkursen verwendet werden können.
```
data['Momentum'] = data['Adj Close'] - data['Adj Close'].shift(4)
```
Das Momentum wird berechnet, indem der aktuelle bereinigte Schlusskurs (Adj Close) mit dem Kurs von vor 4 Tagen verglichen wird. Das Momentum gibt die Stärke einer Kursbewegung an.

```
data['ROC'] = data['Adj Close'].pct_change(periods=10) * 100
```
Als nächstes wird die Rate of Change (ROC) berechnet, die die prozentuale Veränderung des bereinigten Schlusskurses über die letzten 10 Tage angibt. Die ROC hilft, die Geschwindigkeit einer Kursbewegung zu beurteilen.
```
data['PriceChange'] = data['Adj Close'].pct_change() * 100
```
Ein weiterer Indikator ist dietägliche prozentuale Veränderung des bereinigten Schlusskurses berechnet. Diese Kennzahl gibt an, wie stark sich der Kurs an einem bestimmten Tag verändert hat.
```
data['Volatility'] = data['Adj Close'].rolling(window=10).std()
```
die Volatilität wird durch ddes bereinigten Schlusskurses über die letzten 10 Tage berechnet. Die Volatilität ist ein Maß für die Schwankungsbreite des Kurses und gibt an, wie stark der Kurs in einem bestimmten Zeitraum schwankt.

```return data```
 Die Funktion gibt den aktualisierten DataFrame zurück, der nun die vier neuen Spalten für die zusätzlichen Indikatoren enthält.


#### 2.2.5 Generierung vom Kauf- und Verkaufssignale

In [ ]:
def calculate_signal(data, threshold,text):
    data['Signal'] = 0
    data.loc[data['PriceChange'] > threshold, 'Signal'] = 1
    data.loc[data['PriceChange'] < -threshold, 'Signal'] = -1
    if text is not None:
      print("Berechnung der Signal")
      print("\n")
    return data

In der Methode ```calculate_signal``` werden die Preisveränderungen nach dem Schwellenwert (threshold) geschaut, ob dieser über oder unterhalb der schwellenwert ist und dem entsprechend ein Kauf- (1) oder Verkaussignal (-1) generiert wird.

#### 2.2.6 Pipeline der Datenvorbereitung und Anpassung

In [ ]:
def prepare_data_pipeline(data):
    data = calculate_moving_averages(data)
    data = calculate_rsi(data)
    data = calculate_bollinger_bands(data)
    data = calculate_additional_indicators(data)
    data.dropna(inplace=True)
    print("Pipeline der Datenvorbereitung und Anpassung")
    print("--------------------Vollständige Aufbereitung--------------------")
    print(data.head())
    print(data.tail())
    print("-----------------------------------------------------------------")
    print("\n")
    return data

In der Methode ```prepare_data_pipeline(data)``` wird die Heruntergeladene Aktie Vorbereitet in dem alle benötigten Indikatoren dem DataFrame(data) hinzugefügt werden.

## 3. Modelltraining und -bewertung

### 3.1 Vorbereitung der Training- und Testdaten

#### 3.1.1 Trennung der Daten für Training und Test

In [ ]:
def prepare_data(data, train_start, train_end, test_start):
    train_data = data.loc[train_start:train_end].copy()
    test_data = data.loc[test_start:].copy()
    return train_data, test_data

In der Funktion ```prepare_data``` werden die vorhandenen Aktiendaten in zwei separate Datensätze aufgeteilt: einen für das Training des Modells und einen zum Testen des Modells.

Um sicherzustellen, dass die ursprünglichen Daten nicht versehentlich verändert werden, werden Kopien der jeweiligen Zeiträume erstellt und zurückgegeben.

#### 3.1.2 Extrahieren der Features und des Zielwerts

In [ ]:
def extract_features_and_target(data, features):
    X = data[features]
    y = data['Signal']
    return X, y

Im ```extract_features_and_target``` werden die Daten für das Maschinen learning vorbereitet.

### 3.2 Ausführung der Trainingsmodelle

#### 3.2 Trainingsmodell mit Technischen Indikatoren

In [ ]:
def train_model(train_data, features, threshold, random_seed):
    """
    Trainiert ein RandomForest-Modell mit den gegebenen Trainingsdaten und den ausgewählten Features.
    """
    train_data = calculate_signal(train_data, threshold, None)
    X_train, y_train = extract_features_and_target(train_data, features)
    model = RandomForestClassifier(n_estimators=100, random_state=random_seed)
    model.fit(X_train, y_train)
    return model

Die Funktion ```train_model``` trainiert ein leistungsstarkes Random Forest Modell zur Vorhersage von Handelssignalen

In [ ]:
def evaluate_model(test_data, model, features, threshold):
    """
    Bewertet das trainierte Modell mit den Testdaten und gibt die Ergebnisse aus.
    """
    test_data = calculate_signal(test_data, threshold, None)
    X_test, y_test = extract_features_and_target(test_data, features)
    y_pred = model.predict(X_test)
    print("Ergebnisse für RandomForest-Modell:")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}\n")

Die Funktion ```evaluate_model``` überprüft die Genauigkeit des trainierten Random Forest Modells.

#### 3.3 Trainingsmodell mit Rohdaten

In [ ]:
# Funktion zum Training und zur Vorhersage mit Rohdaten
def train_and_predict_raw_data(data, threshold, train_start, train_end, test_start, raw_features, random_seed,verbose=False ):



    data = calculate_signal(data, threshold,"für Rohdaten")
    data.dropna(inplace=True)

    train_data, test_data = prepare_data(data, train_start, train_end, test_start)

    X_train_raw, y_train_raw = extract_features_and_target(train_data, raw_features)
    X_test_raw, y_test_raw = extract_features_and_target(test_data, raw_features)

    scaler = StandardScaler()
    X_train_raw = scaler.fit_transform(X_train_raw)
    X_test_raw = scaler.transform(X_test_raw)

    model_raw = LogisticRegression(random_state = random_seed)

    model_raw.fit(X_train_raw, y_train_raw)

    y_pred_raw = model_raw.predict(X_test_raw)

    if verbose:
        print("Ergebnisse für Rohdaten-Modell:")
    print(classification_report(y_test_raw, y_pred_raw))
    print("Accuracy:", accuracy_score(y_test_raw, y_pred_raw))

    return model_raw, scaler

Die Funktion ```train_and_predict_raw_data``` trainiert ein zusätzliches Modell basierend auf den Rohdaten
```(OHLCV - Open, High, Low, Close, Volume)``` der Aktie. Dieses Modell liefert eine alternative Perspektive und kann die Vorhersagegenauigkeit weiter verbessern.

## 4. Vorhersagen


### 4.1 Vorhersage mit technischen Indikatoren

In [ ]:
# Funktion zur Vorhersage für ein Datum mit RandomForest-Modell
def predict_for_date(prediction_date, data, model, features):
    start_date = prediction_date - timedelta(days=10)
    end_date = prediction_date - timedelta(days=1)
    last_10_days = data.loc[start_date:end_date]
    if last_10_days.empty:
        return None, None, None, None

    last_10_days_features = last_10_days[features].tail(1)

    prediction = model.predict(last_10_days_features)

    if prediction_date in data.index:
        prediction_price = data.loc[prediction_date]['Adj Close']
    else:
        prediction_price = None

    next_day = prediction_date + timedelta(days=1)
    if next_day in data.index:
        next_day_price = data.loc[next_day]['Adj Close']
    else:
        next_day_price = None

    prediction_label = 'Kauf' if prediction[0] == 1 else 'Verkauf' if prediction[0] == -1 else 'Halten'

    return prediction_date, prediction_price, next_day_price, prediction_label

Die Funktion ```predict_for_date``` ermöglicht es, gezielt für ein bestimmtes Datum ein Handelssignal zu generieren. Anhand der Kursdaten der letzten 10 Tage wird eine Vorhersage getroffen, ob die Aktie an dem gewählten Tag gekauft, gehalten oder verkauft werden sollte.

### 4.2 Vorhersage mit Rohdaten

In [ ]:
# Funktion zur Vorhersage für ein Datum mit Rohdaten-Modell
def predict_for_date_raw(prediction_date, data, model, scaler):
    start_date = prediction_date - timedelta(days=10)
    end_date = prediction_date - timedelta(days=1)

    last_10_days = data.loc[start_date:end_date]
    if last_10_days.empty:
        return None, None, None, None

    last_10_days_raw_features = last_10_days[['Open', 'High', 'Low', 'Close', 'Volume']].tail(1)
    last_10_days_raw_features = scaler.transform(last_10_days_raw_features)

    prediction = model.predict(last_10_days_raw_features)

    if prediction_date in data.index:
        prediction_price = data.loc[prediction_date]['Adj Close']
    else:
        prediction_price = None

    next_day = prediction_date + timedelta(days=1)
    if next_day in data.index:
        next_day_price = data.loc[next_day]['Adj Close']
    else:
        next_day_price = None

    prediction_label = 'Kauf' if prediction[0] == 1 else 'Verkauf' if prediction[0] == -1 else 'Halten'

    return prediction_date, prediction_price, next_day_price, prediction_label


Die Funktion ```predict_for_date_raw``` nutzt die Rohdaten (OHLCV) der Aktie, um gezielt für ein bestimmtes Datum ein Handelssignal (Kauf, Halten, Verkauf) zu generieren.

### 4.3 Vorhersage für die Simulation trading

#### 4.3.1 Handelssignale für einen Zeitraum generieren

In [ ]:
def generate_predictions(data, model, model_raw, scaler, features, start_date, end_date):
    results_rf = []
    results_raw = []
    current_date = start_date

    while current_date <= end_date:
        rf_result = predict_for_date(current_date, data, model, features)
        raw_result = predict_for_date_raw(current_date, data, model_raw, scaler)

        if rf_result[0] is not None and rf_result[1] is not None:
            results_rf.append(rf_result)
        if raw_result[0] is not None and raw_result[1] is not None:
            results_raw.append(raw_result)

        current_date += timedelta(days=1)

    return results_rf, results_raw

Die Funktion ```generate_predictions``` berechnet Handelssignale für jeden Tag innerhalb eines bestimmten Zeitraums, sowohl basierend auf technischen Indikatoren als auch auf Rohdaten. Diese Signale dienen als Grundlage für die spätere Simulation und Bewertung der Handelsstrategie.

#### 4.3.2 Vorhersagen für 2022 durchführen und speichern

In [ ]:
def run_predictions(data, model, model_raw, scaler, features):
    """
    Generiert Vorhersagen für das Jahr 2022, formatiert die Ergebnisse und speichert sie.
    """
    start_date_2022 = datetime(2022, 1, 1)
    end_date_2022 = datetime(2022, 12, 31)

    results_rf, results_raw = generate_predictions(data, model, model_raw, scaler, features, start_date_2022, end_date_2022)
    df_rf, df_raw = format_results(results_rf, results_raw)

    save_results(df_rf, df_raw)
    return df_rf, df_raw


Die Funktion ```run_predictions``` führt den gesamten Prozess der Erstellung, Formatierung und Speicherung von Handelssignalen für das Jahr 2022 durch.

#### 4.3.3 Vorhersageergebnisse optimieren

In [ ]:
def format_results(results_rf, results_raw):
    """
    Formatiert die Vorhersageergebnisse und bereitet sie für die Speicherung vor.
    """
    df_rf = pd.DataFrame(results_rf, columns=['Datum', 'Preis', 'Nächster Tag Preis', 'Vorhersage_RF'])
    df_raw = pd.DataFrame(results_raw, columns=['Datum', 'Preis', 'Nächster Tag Preis', 'Vorhersage_Raw'])

    df_rf = df_rf.drop(columns=['Nächster Tag Preis']).dropna(subset=['Preis'])
    df_raw = df_raw.drop(columns=['Nächster Tag Preis']).dropna(subset=['Preis'])

    return df_rf, df_raw

Die Funktion ```format_results``` bereitet die generierten Vorhersageergebnisse für die spätere Verwendung auf, indem sie unnötige Spalten entfernt und fehlende Werte behandelt.

#### 4.3.4 Speicherung der Vohersageergebnisse

In [ ]:
def save_results(df_rf, df_raw):
    """
    Speichert die formatierten Ergebnisse als CSV-Dateien.
    """
    df_rf.to_csv('results_rf_2022.csv', index=False)
    df_raw.to_csv('results_raw_2022.csv', index=False)
    print("CSV-Dateien für das Jahr 2022 wurden gespeichert.\n")

Die Funktion ```save_results``` speichert die aufbereiteten Vorhersageergebnisse in CSV-Dateien, um sie dauerhaft zu sichern und für weitere Analysen verfügbar zu machen.

## 5. Simulation

### 5.1 Handelsstrategie simulieren

In [ ]:
# Funktion zur Durchführung der Simulation
def simulate_trading(df, initial_balance=1000, verbose=False):
    balance = initial_balance
    holdings = 0
    last_action = None
    last_action_date = None

    results = []

    for index, row in df.iterrows():
        date = row['Datum']
        price = row['Preis']
        action = row['Vorhersage_RF'] if 'Vorhersage_RF' in row else row['Vorhersage_Raw']

        if action == 'Kauf' and (last_action != 'Kauf' or last_action_date is None or (date - last_action_date).days > 1):
            if balance > 0:
                holdings = balance / price
                balance = 0
                last_action = 'Kauf'
                last_action_date = date
                if verbose:
                    print(f"{date}: Kauf für {price:.2f}. Guthaben: {balance:.2f}, Bestände: {holdings:.2f} Aktien")

        elif action == 'Verkauf' and (last_action != 'Verkauf' or last_action_date is None or (date - last_action_date).days > 1):
            if holdings > 0:
                balance = holdings * price
                holdings = 0
                last_action = 'Verkauf'
                last_action_date = date
                if verbose:
                    print(f"{date}: Verkauf für {price:.2f}. Guthaben: {balance:.2f}, Bestände: {holdings:.2f} Aktien")

        results.append({
            'Datum': date,
            'Guthaben': balance,
            'Bestände': holdings * price if holdings > 0 else 0
        })

    # Endwert berechnen
    end_value = balance + holdings * price if holdings > 0 else balance

    # Prozentualen Gewinn/Verlust berechnen
    profit_loss_percent = ((end_value - initial_balance) / initial_balance) * 100

    if verbose:
        print(f"Endwert der Simulation: {end_value:.2f} Euro")
        print(f"Prozentualer Gewinn/Verlust: {profit_loss_percent:.2f}%")

    return balance, end_value, profit_loss_percent


Die Funktion ```simulate_trading``` simuliert den Aktienhandel basierend auf den generierten Handelssignalen. Mit einem virtuellen Startkapital werden Kauf- und Verkaufsentscheidungen über einen bestimmten Zeitraum simuliert. Das Ergebnis der Simulation gibt Aufschluss über die potenzielle Performance der Handelsstrategie.

### 5.2 Auswertung der Simulation

In [ ]:
def simulate_and_evaluate(df_rf, df_raw):
    """
    Führt die Simulation von Trading-Strategien durch und speichert die Ergebnisse.
    """
    print("Simulation für technischer Indikator:")
    start_balance_rf, end_balance_rf, profit_loss_percent_rf = simulate_trading(df_rf, verbose=False)
    print(f"Simulation technischer Indikator: Start: 1000€, Ende: {end_balance_rf:.2f}€, Gewinn/Verlust: {profit_loss_percent_rf:.2f}%")
    pd.DataFrame([{'Start': 1000, 'Ende': end_balance_rf, "Gewinn/Verlust": profit_loss_percent_rf}]).to_csv('simulation_rf_2022_summary.csv', index=False)
    print("Auswertung wurde im Datei simulation_rf_2022_summary.csv aufgezeichnet\n")

    print("Simulation für Rohdaten-Indikator:")
    start_balance_raw, end_balance_raw, profit_loss_percent_raw = simulate_trading(df_raw, verbose=False)
    print(f"Simulation Rohdaten-Indikator: Start: 1000€, Ende: {end_balance_raw:.2f}€, Gewinn/Verlust: {profit_loss_percent_raw:.2f}%")
    pd.DataFrame([{'Start': 1000, 'Ende': end_balance_raw, "Gewinn/Verlust": profit_loss_percent_raw}]).to_csv('simulation_raw_2022_summary.csv', index=False)
    print("Auswertung wurde im Datei simulation_raw_2022_summary.csv aufgezeichnet\n")


Die Funktion ```simulate_and_evaluate``` führt die Simulation von Handelsstrategien für sowohl das Random Forest Modell als auch das Rohdaten-Modell durch und bewertet deren Performance anhand von Kennzahlen wie dem Endsaldo und dem prozentualen Gewinn/Verlust.

## 6. Interaktion mit dem Benutzer

### 6.1 Eingabe der Aktie

In [ ]:
# Funktion zur Eingabe des Datums für die Vorhersage
def get_prediction_date():
    while True:
        try:
            user_input = input("Bitte geben Sie das Datum für die Vorhersage im Format 'YYYY-MM-DD' ein: ")
            prediction_date = datetime.strptime(user_input, '%Y-%m-%d')
            return prediction_date
        except ValueError:
            print("Ungültiges Datum. Bitte geben Sie das Datum im Format 'YYYY-MM-DD' ein.")


### 6.2 Darstellung der Vorhersage

In [ ]:
def display_predictions(prediction_date, data, model, model_raw, scaler, features):
    """
    Zeigt Vorhersagen für ein spezifisches Datum an.
    """
    pred_date, pred_price, next_day_price, pred_label = predict_for_date(prediction_date, data, model, features)
    if pred_date:
        if pred_price is not None:  # Check if pred_price is valid
            print(f"Aktien Preis am {pred_date.strftime('%Y-%m-%d')} = ${pred_price:.2f}")
        else:
            print(f"Keine Preisvorhersage verfügbar für {pred_date.strftime('%Y-%m-%d')}.")  # Inform the user
        if next_day_price is not None:  # Check if next_day_price is not None
            print(f"Aktien Preis für den darauffolgenden Tag = ${next_day_price:.2f}")
        else:
            print("Aktien Preis für den darauffolgenden Tag konnte nicht berechnet werden.")
        print(f"Vorhersage für {pred_date.strftime('%Y-%m-%d')}: Vorhersage = {pred_label}")
    else:
        print(f"Keine Vorhersage möglich für {prediction_date.strftime('%Y-%m-%d')}.")

    pred_date_raw, pred_price_raw, next_day_price_raw, pred_label_raw = predict_for_date_raw(prediction_date, data, model_raw, scaler)
    if pred_date_raw:
        print(f"Rohdaten-Vorhersage für {pred_date_raw.strftime('%Y-%m-%d')}: Vorhersage = {pred_label_raw}")
    else:
        print(f"Keine Rohdaten-Vorhersage möglich für {prediction_date.strftime('%Y-%m-%d')}.")

Die Funktion ```display_predictions``` zeigt die generierten Handelssignale für ein bestimmtes Datum übersichtlich an, sowohl für das Random Forest Modell als auch für das Rohdaten-Modell.

### 6.3 Eingabe der Vorhersage für einen Bestimmten tag

In [ ]:
def interact_with_user(data, model, model_raw, scaler, features):
    """
    Interagiert mit dem Benutzer, um Vorhersagen für spezifische Daten bereitzustellen.
    """
    while True:
        verbose = input("Möchten Sie detaillierte Daten der letzten 10 Tage anzeigen? (ja/nein): ").strip().lower() == 'ja'
        prediction_date = get_prediction_date()
        display_predictions(prediction_date, data, model, model_raw, scaler, features)

        another_day = input("Möchten Sie ein weiteres Datum eingeben? (ja/nein): ").strip().lower()
        if another_day != 'ja':
            print("Programm beendet.")
            break


Die Funktion ```interact_with_user``` ermöglicht dem Benutzer, interaktiv mit dem Programm zu interagieren, indem er ein Datum eingibt und die dazugehörigen Vorhersagen erhält.

### 6.4 Auswahl zwischen Simulation und Vorhersage für ein bestimmten Tag

In [ ]:
def handle_user_choice(df_rf, df_raw, data, model, model_raw, scaler, features):
    """Fragt den Benutzer nach seiner Auswahl und führt die entsprechende Aktion aus."""
    while True:
        choice = input("Was möchten Sie tun? (1 für Simulation, 2 für Vorhersage): ")
        if choice == '1':
            simulate_and_evaluate(df_rf, df_raw)

            # Abfrage, ob die Simulation erneut ausgeführt werden soll
            repeat_simulation = input("Möchten Sie die Simulation erneut ausführen? (ja/nein): ").strip().lower()
            if repeat_simulation != 'ja':
                break  # Schleife verlassen, wenn keine Wiederholung gewünscht ist

        elif choice == '2':
            interact_with_user(data, model, model_raw, scaler, features)
            break
        else:
            print("Ungültige Auswahl. Bitte wählen Sie 1 oder 2.")

## 7. Hauptfunktion


In [ ]:
def main():
    # Globale Parameter initialisieren und zurückgeben
    train_start, train_end, test_start, threshold, features, raw_features, random_seed = initialize_globals()

    # Stock-Daten abrufen und Datenpipeline durchlaufen
    data = get_stock_data()
    data = prepare_data_pipeline(data)

    # Daten aufteilen
    train_data, test_data = prepare_data(data, train_start, train_end, test_start)

    # Modell trainieren und evaluieren für
    model = train_model(train_data, features, threshold, random_seed)
    evaluate_model(test_data, model, features, threshold)

    # Rohdaten-Modell trainieren und Vorhersagen generieren
    model_raw, scaler = train_and_predict_raw_data(data, threshold, train_start, train_end, test_start, raw_features, random_seed,verbose=False)

    # Vorhersagen für das Jahr 2022 ausführen und speichern
    df_rf, df_raw = run_predictions(data, model, model_raw, scaler, features)

    # Auswahl verarbeiten
    handle_user_choice(df_rf, df_raw, data, model, model_raw, scaler, features)


Die Main Methode Ruft alle Methoden nach der gewünschten Rheinfolge aus!

In [ ]:
if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed




Aktie: Apple Inc. wurde erfolgreich Heruntergeladen
Anzahl der Daten: 3692




Berechnung der Moving Averages MA5 und MA10


Berechnung des Relative Strength Index(RSI)


Berechnung des Bollinger Bänder


Berechnung weiterer Indikatoren


Pipeline der Datenvorbereitung und Anpassung
--------------------Vollständige Aufbereitung--------------------
                Open      High       Low     Close  Adj Close     Volume  \
Date                                                                       
2010-02-01  6.870357  7.000000  6.832143  6.954643   5.873024  749876400   
2010-02-02  6.996786  7.011429  6.906429  6.995000   5.907104  698342400   
2010-02-03  6.970357  7.150000  6.943571  7.115357   6.008744  615328000   
2010-02-04  7.026071  7.084643  6.841786  6.858929   5.792197  757652000   
2010-02-05  6.879643  7.000000  6.816071  6.980714   5.895041  850306800   

                 MA5      MA10        RSI  MiddleBand  UpperBand  LowerBand  \
Date                                

**Erkenntnis:**


Die Simulation zeigt, dass technische Indikatoren im Zusammenhang mit den analysierten Aktienkursen in den bisherigen Tests eine bessere Performance erzielen. Allerdings sind die Ergebnisse nicht endgültig, da sich bei erneuter Durchführung der Simulation Schwankungen zeigen. Dies deutet darauf hin, dass eine Optimierung des **RandomForestClassifiers** notwendig ist, um robustere und konsistentere Vorhersagen zu erhalten. Darüber hinaus sollten weitere Modelle betrachtet werden, um eine umfassendere Analyse der Kursentwicklung zu ermöglichen und einen  Vergleich zwischen den Modellen zu *erzielen*.